# Assignment 1: Data Sharing with Encryption
### Security and Privacy Project - MECD (2022/2023)

### Autores

- Duarte Meneses - 2019216949
- Patricia Costa - 2019213995

### **ControlerER**

In [ ]:
%pip install cryptography
!pip uninstall pycrypto -y
!pip3 install pycryptodome pmlb

In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.serialization import Encoding, ParameterFormat, PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_parameters, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

### Key exchange

In [ ]:
%%time
#1

parameters = dh.generate_parameters(generator=2, key_size=1024, backend=default_backend())

with open("shared_file", "wb") as file:
    file.write(parameters.parameter_bytes(encoding=Encoding.PEM, format=ParameterFormat.PKCS3))

In [ ]:
%%time
#3

with open ("shared_file", "rb") as file:
    public_mixed_with_D = load_pem_public_key(file.read()) 

private_key_C = parameters.generate_private_key()
public_mixed_with_C  = private_key_C.public_key()

with open("shared_file", "wb") as file:
    file.write(public_mixed_with_C.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

In [ ]:
%%time
#6

shared_key_D_then_C = private_key_C.exchange(public_mixed_with_D)

derived_key_D_then_C = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_D_then_C)

### Encryption without Integrity and Authenticity 

In [14]:
from Crypto.Cipher import AES
from Crypto.Util import Counter

In [ ]:
%%time
#1

def encrypt_csv_ctr(file_name, civ, counter_size=128):
    
    counter = Counter.new(counter_size, initial_value=civ)
    
    aes = AES.new(derived_key_D_then_C, mode=AES.MODE_CTR, counter=counter)

    with open(file_name, 'rb') as file:
        csv_data = file.read()
        encrypted_csv = aes.encrypt(csv_data)
    return encrypted_csv

columns_desc_enc = encrypt_csv_ctr("data/columns_description-CORRECTED.csv", 10)

with open("shared_file", "wb") as file:
    file.write(columns_desc_enc)

In [ ]:
%%time
#4

infringement_dataset_enc = encrypt_csv_ctr("data/infringement_dataset.csv", 10)

with open("shared_file", "wb") as file:
    file.write(infringement_dataset_enc)

### Encryption with Integrity and Authenticity 

In [6]:
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

In [ ]:
%%time
#1

with open("data/columns_description-CORRECTED.csv", 'rb') as file:
        columns_desc = file.read()

aesgcm = AESGCM(derived_key_D_then_C)

nonce = os.urandom(12)
columns_desc_enc = aesgcm.encrypt(nonce, columns_desc, None)

with open("shared_file", "wb") as file:
    file.write(columns_desc_enc)

In [ ]:
%%time
#3

with open("shared_file", "wb") as file:
    file.write(nonce)

In [ ]:
%%time
#5

with open("data/infringement_dataset.csv", 'rb') as file:
        infringement_dataset = file.read()

nonce = os.urandom(12)
infringement_dataset_enc = aesgcm.encrypt(nonce, infringement_dataset, None) # your code here

with open("shared_file", "wb") as file:
    file.write(infringement_dataset_enc)

In [ ]:
%%time
#7

with open("shared_file", "wb") as file:
    file.write(nonce)